Packages:

- [NLopt.jl](https://github.com/JuliaOpt/NLopt.jl) Powell's derivative-free algorithms
- [Convex.jl](https://github.com/JuliaOpt/Convex.jl) Disciplined convex programming
- [GLPK.jl](https://github.com/JuliaOpt/GLPK.jl) Open source linear programming solver

In [1]:
using NLopt
using Convex
using GLPK
using Printf
using Optim
using FiniteDiff

# Computing the local visibility of quantum correlations

We say that the correlations $P_\text{AB|ST}(a,b|s,t)$ have a local model if there is a joint distribution $P_{\text{A}_1,\text{A}_2,\text{B}_1,\text{B}_2}(a_1,a_2,b_1,b_2)$ such that

$$ P_\text{AB|ST}(a,b|s,t) = \sum_{a_1,a_2,b_1,b_2} \delta_{a, a_s} \delta_{b, b_t} P_{\text{A}_1,\text{A}_2,\text{B}_1,\text{B}_2}(a_1,a_2,b_1,b_2) $$

The distribution $P_\text{AB|ST}$ can be seen as a vector $\vec{P}_\text{Q} \in \mathbb{R}^{16}$ by a suitable enumeration of coefficients (start by incrementing $a$, then $b$, then $s$, then $t$, as with [column-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order) used in Julia).

We also enumerate the joint distribution $P_{\text{A}_1,\text{A}_2,\text{B}_1,\text{B}_2}$ as a vector $\vec{y} \in \mathbb{R}^{16}$.
 
Then $\vec{P}_\text{Q}$ has a local model if there exists $\vec{y}$ with $y_i \ge 0, \forall i$, and $\sum_i y_i = 1$, such that

$$ \vec{P}_\text{Q} = M \cdot \vec{y}, $$

where the matrix $M$ encodes the relationship between $P_\text{AB|ST}$ and $P_{\text{A}_1,\text{A}_2,\text{B}_1,\text{B}_2}$ as described above.

A crude but reasonable model of noise in experiments is given by the uniformly random distribution $P_\text{AB|ST}(a,b|s,t) = 1/4$, i.e. $\vec{P}_0 = (1, \ldots, 1)^\top / 4$.

Given quantum correlations described by $\vec{P}_\text{Q}$, we define the noisy correlations:

$$ \vec{P}_v = v \vec{P}_\text{Q} + (1-v) \vec{P}_0. $$

The *visibility* of correlations is defined as the following maximization problem:

$$ v^*(\vec{P}_\text{Q}) = \max_{v \in \mathbb{R}, \vec{y} \in \mathbb{R}^{16}} v \qquad \text{s.t.} \qquad v \vec{P}_\text{Q} + (1-v) \vec{P}_0 = M \vec{y}, \quad v \in [0,1], \quad \vec{y} \ge 0, \quad \sum_i y_i = 1.$$


In [2]:
function visibility(P)
# a,b,s,t,a1,a2,b1,b2
    M = zeros((2,2,2,2,2,2,2,2))
    for a = 1:2
        for b = 1:2
            for s = 1:2
                for t = 1:2
                    for a1 = 1:2
                        for a2 = 1:2
                            for b1 = 1:2
                                for b2 = 1:2
                                    a12 = [a1, a2]
                                    b12 = [b1, b2]
                                    if a == a12[s] && b == b12[t]
                                        M[a,b,s,t,a1,a2,b1,b2] = 1
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    P = reshape(P, 16)
    M = reshape(M, (16, 16))
    v = Variable(1);
    y = Variable(16);
    P0 = ones(16)/4;
    constraints = [v >= 0, v <= 1, v*P + (1-v)*P0 == M*y, y >= 0, sum(y) == 1]
    problem = Convex.maximize(v, constraints)
    solve!(problem, GLPK.Optimizer)
    return v.value
end

visibility (generic function with 1 method)

In [3]:
function singlet_proj_prob(x::Vector)
# variable x(1:4) Alice angles, x(5:8) Bob angles
    P = zeros(ComplexF64, (2, 2, 2, 2));
    A = zeros(ComplexF64, (2, 2, 2));
    B = zeros(ComplexF64, (2, 2, 2));
    A[:,1,1] = [ cos(x[1])
                 sin(x[1])*exp(1im*x[2])];
    A[:,2,1] = [-sin(x[1])
                 cos(x[1])*exp(1im*x[2])];
    A[:,1,2] = [ cos(x[3])
                 sin(x[3])*exp(1im*x[4])];
    A[:,2,2] = [-sin(x[3])
                 cos(x[3])*exp(1im*x[4])];
    B[:,1,1] = [ cos(x[5])
                 sin(x[5])*exp(1im*x[6])];
    B[:,2,1] = [-sin(x[5])
                 cos(x[5])*exp(1im*x[6])];
    B[:,1,2] = [ cos(x[7])
                 sin(x[7])*exp(1im*x[8])];
    B[:,2,2] = [-sin(x[7])
                 cos(x[7])*exp(1im*x[8])];
    for s = 1:2
        for t = 1:2
            for a = 1:2
                for b = 1:2
                    ov = kron(A[:,a,s], B[:,b,t])' * [0; 1; -1; 0]/sqrt(2);
                    P[a,b,s,t] = real(conj(ov)*ov);
                end
            end
        end
    end
    return P
end

singlet_proj_prob (generic function with 1 method)

In [4]:
P = singlet_proj_prob([0.8048392485916049, 2.8093287208324975, 4.9007496908847985, 1.3370237012924213, 1.9788869048686093, 6.314181068258849, 0.44459933293709847, 2.4694786009799614])
visibility(P)

0.7071067811865476

# Bilevel optimization

[General overview](https://en.wikipedia.org/wiki/Bilevel_optimization)

We now parameterize $\vec{P}_\text{Q}$ using the angles $\vec{x} \in \mathbb{R}^8$, and want to find the measurement angles that lead to the minimal local visibility:

$$\vec{x}^* = \operatorname{arg} \min_\vec{x} v^*(\vec{P}^{(\vec{x})}_\text{Q})$$

In [5]:
## Using NL
function vis(x::Vector, grad::Vector)
   val = visibility(singlet_proj_prob(x))
   return val
end

x0 = rand(8)*2*pi
opt = Opt(:LN_BOBYQA, 8)
opt.lower_bounds = zeros(8)*1.0
opt.upper_bounds = ones(8)*2*pi*1.0
opt.min_objective = vis
opt.xtol_rel = 1e-4
(minf, minx, ret) = NLopt.optimize(opt, x0)
@printf("\n\nMinimum: %f in %d iterations\n", minf, opt.numevals)
@printf("Diff: %0.1E\n", abs(minf - 1/sqrt(2)))



Minimum: 0.707107 in 141 iterations
Diff: 1.5E-09


# Exercices

- Run the algorithm a number of times, how many times does it converge to the best maximum?
- Scan through the family of states $\left| \gamma \right> = \cos \gamma \left| 01 \right> + \sin \gamma \left| 10 \right>$, plot the best visibility for each $\gamma$.